In [6]:
!pip3 install -q mediapipe

You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [6]:
!pip3 install -q imageio

You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [ ]:
import cv2
import mediapipe as mp
import os
import numpy as np
# warning levels 
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
# "0" = all logs (default)
# "1" = filter out INFO
# "2" = filter out INFO + WARNING
# "3" = filter out INFO + WARNING + ERROR 

In [ ]:
data_folder = '../data/'
signs = ['hello', 'bye', 'world']
videos_folders = [f for f in os.listdir(data_folder) if os.path.isdir(os.path.join(data_folder, f)) and any(nombre in f for nombre in signs)]
print(videos_folders)
video_files = [ os.path.join(folder, f) for folder in videos_folders for f in os.listdir(os.path.join(data_folder, folder)) if f.endswith(('.mp4', '.avi', '.mov'))]
print(video_files)
MODEL_PATH = "hand_landmarker.task"  

['world', 'bye', 'hello']
['world/Screen Recording 2025-03-17 at 19.51.51.mov', 'world/Screen Recording 2025-03-17 at 19.52.17.mov', 'world/Screen Recording 2025-03-17 at 19.52.38.mov', 'world/Screen Recording 2025-03-17 at 19.50.29.mov', 'world/Liz-session-ASL_2008_03_28-scene-44-2671-2727-camera1.mp4', 'world/Screen Recording 2025-03-17 at 19.51.21.mov', 'world/Screen Recording 2025-03-17 at 19.49.49.mov', 'world/Brady-session-ASL_2011_07_22_Brady-scene-33-597-631-camera1.mp4', 'world/Screen Recording 2025-03-17 at 19.49.04.mov', 'bye/Brady-session-ASL_2011_07_22_Brady-scene-25-2180-2221-camera1.mp4', 'bye/Screen Recording 2025-03-17 at 19.43.55.mov', 'bye/Screen Recording 2025-03-17 at 19.44.25.mov', 'bye/Screen Recording 2025-03-17 at 19.42.41.mov', 'bye/Screen Recording 2025-03-17 at 19.45.38.mov', 'bye/Liz-session-ASL_2008_03_28-scene-37-1435-1485-camera1.mp4', 'bye/Screen Recording 2025-03-17 at 19.44.52.mov', 'bye/Screen Recording 2025-03-17 at 19.45.17.mov', 'bye/Brady-session

## Extract Keypoints

In [ ]:

# Inicializar MediaPipe Holistic para detectar postura y manos en una imagen o video.
mp_holistic = mp.solutions.holistic

# Configuaracion de carpetas 
video_folder = "../data/hello/"  # Asegúrate de definir la carpeta correctamente
output_folder = os.path.join(video_folder, "keypoints")
os.makedirs(output_folder, exist_ok=True)

# Número esperado de keypoints
POSE_LANDMARKS = 33
HAND_LANDMARKS = 21
TOTAL_KEYPOINTS = POSE_LANDMARKS + 2 * HAND_LANDMARKS  # Cuerpo + 2 manos

# Obtener los archivos de video
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]
if not video_files:
    print("⚠️ No se encontraron archivos de video en la carpeta.")
else:
    print(f"{len(video_files)} videos encontrados. Procesando...")

# Procesar cada video
for video_name in video_files:
    video_path = os.path.join(video_folder, video_name)
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"⚠️ No se pudo abrir el video: {video_name}")
        continue

    keypoints_list = []
    print(f"Procesando: {video_name}")

    # Procesamiento de fotogramas con media pipe
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Convertir a RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Detectar puntos clave
            results = holistic.process(frame_rgb)

            # Inicializar keypoints con ceros
            keypoints = [(0, 0, 0)] * TOTAL_KEYPOINTS

            if results.pose_landmarks:
                print('🍑 pose landmarks detected')
                for i, lm in enumerate(results.pose_landmarks.landmark):
                    keypoints[i] = (lm.x, lm.y, lm.z)

            if results.left_hand_landmarks:
                print('🌶️ left hand landmarks detected')
                for i, lm in enumerate(results.left_hand_landmarks.landmark):
                    keypoints[POSE_LANDMARKS + i] = (lm.x, lm.y, lm.z)

            if results.right_hand_landmarks:
                print('🫐 right hand landmarks detected')
                for i, lm in enumerate(results.right_hand_landmarks.landmark):
                    keypoints[POSE_LANDMARKS + HAND_LANDMARKS + i] = (lm.x, lm.y, lm.z)

            keypoints_list.append(keypoints)

        cap.release()

    if keypoints_list:
        # Convertir a array NumPy y guardar
        keypoints_array = np.array(keypoints_list)
        output_path = os.path.join(output_folder, video_name.replace(".mp4", ".npy"))
        np.save(output_path, keypoints_array)
        print(f"✅ Keypoints guardados en {output_path}")
    else:
        print(f"⚠️ No se extrajeron keypoints para {video_name}")

print("Proceso finalizado.")


9 videos encontrados. Procesando...
Procesando: Tyler-session-ASL_2008_05_29a-scene-49-700-717-camera1.mp4
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected


I0000 00:00:1742563673.525738 1804077 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M4 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1742563673.582783 1804256 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742563673.590935 1804256 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742563673.592537 1804256 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742563673.592635 1804258 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742563673.592725 1804257 inference_feedback_manager.cc:114] Feedback manager requires a

🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose lan

I0000 00:00:1742563674.036913 1804077 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M4 Pro
W0000 00:00:1742563674.090306 1804293 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742563674.098376 1804301 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742563674.099432 1804301 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742563674.099431 1804300 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742563674.099481 1804295 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling supp

🍑 pose landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🫐 right hand landmarks detected
🍑 pose landmar

...

🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose lan

I0000 00:00:1742563678.004220 1804077 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M4 Pro
W0000 00:00:1742563678.056524 1804471 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742563678.064115 1804473 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742563678.065060 1804478 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742563678.065115 1804470 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742563678.065136 1804475 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling supp

🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose lan

I0000 00:00:1742563678.579467 1804077 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M4 Pro
W0000 00:00:1742563678.632265 1804497 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742563678.640132 1804497 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742563678.641113 1804498 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742563678.641114 1804496 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742563678.641224 1804500 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling supp

🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose landmarks detected
🌶️ left hand landmarks detected
🫐 right hand landmarks detected
🍑 pose lan

### Visualize Extracted Keypoints

In [3]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

# Folder where keypoint files are stored
keypoints_folder = "../data/hello/keypoints"

# Function to visualize keypoints from a .npy file
def visualize_keypoints(npy_file):
    keypoints = np.load(npy_file)  # Load keypoints data
    num_frames = keypoints.shape[0]

    print(f"Visualizing {os.path.basename(npy_file)} ({num_frames} frames)")

    # Create a blank image for visualization
    img_size = 500
    blank_frame = np.ones((img_size, img_size, 3), dtype=np.uint8) * 255

    # Define landmarks (POSE = 33, HAND = 21 per hand)
    POSE_LANDMARKS = 33
    HAND_LANDMARKS = 21
    TOTAL_KEYPOINTS = POSE_LANDMARKS + 2 * HAND_LANDMARKS  # Pose + 2 Hands

    # Animate the keypoints frame by frame
    for frame_idx in range(num_frames):
        img = blank_frame.copy()
        frame_keypoints = keypoints[frame_idx]

        # Normalize & scale keypoints to fit the image
        scaled_keypoints = (frame_keypoints[:, :2] * img_size).astype(int)

        # Draw pose landmarks
        for i in range(POSE_LANDMARKS):
            x, y = scaled_keypoints[i]
            cv2.circle(img, (x, y), 3, (0, 0, 255), -1)  # Red for pose

        # Draw left hand landmarks
        for i in range(HAND_LANDMARKS):
            x, y = scaled_keypoints[POSE_LANDMARKS + i]
            cv2.circle(img, (x, y), 3, (255, 0, 0), -1)  # Blue for left hand

        # Draw right hand landmarks
        for i in range(HAND_LANDMARKS):
            x, y = scaled_keypoints[POSE_LANDMARKS + HAND_LANDMARKS + i]
            cv2.circle(img, (x, y), 3, (0, 255, 0), -1)  # Green for right hand

        # Show the frame
        cv2.imshow("Keypoints Visualization", img)
        if cv2.waitKey(50) & 0xFF == ord('q'):  # Press 'q' to quit
            break

    cv2.destroyAllWindows()
    print(" Visualization complete.")

# Get all .npy files for keypoints
npy_files = [f for f in os.listdir(keypoints_folder) if f.endswith(".npy")]

if not npy_files:
    print("⚠️ No keypoints found.")
else:
    print(f"Found {len(npy_files)} keypoint files.")

    # Visualize the first keypoint file (change index for other files)
    visualize_keypoints(os.path.join(keypoints_folder, npy_files[0]))
    # Force window close
    cv2.waitKey(1)
    cv2.destroyAllWindows()

Found 9 keypoint files.
Visualizing Naomi-session-ASL_2008_08_13-scene-10-1822-1842-camera1.npy (21 frames)


2025-03-21 14:28:09.359 Python[34087:1804077] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-21 14:28:09.359 Python[34087:1804077] +[IMKInputSession subclass]: chose IMKInputSession_Modern


 Visualization complete.


## Extract Start & End Frames Using Keypoints

In [4]:
import cv2
import mediapipe as mp
import os
import numpy as np

# paths & folders
video_folder = "../data/hello"
output_folder = os.path.join(video_folder, "keypoints")
frame_output_folder = os.path.join(video_folder, "key_frames")
os.makedirs(output_folder, exist_ok=True)
os.makedirs(frame_output_folder, exist_ok=True)

# MediaPipe setup
mp_holistic = mp.solutions.holistic

# n of keypoints
POSE_LANDMARKS = 33
HAND_LANDMARKS = 21
TOTAL_KEYPOINTS = POSE_LANDMARKS + 2 * HAND_LANDMARKS  # Body + 2 hands

# Process video files
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

if not video_files:
    print("⚠️ No se encontraron archivos de video en la carpeta.")
else:
    print(f"{len(video_files)} videos encontrados. Procesando...")

# Function to calculate total movement between frames
def calculate_movement(prev_kp, curr_kp):
    if prev_kp is None or curr_kp is None:
        return 0  # No movement detected if landmarks are missing

    total_movement = np.sum(np.linalg.norm(np.array(curr_kp) - np.array(prev_kp), axis=1))
    return total_movement

# Process each video
for video_name in video_files:
    video_path = os.path.join(video_folder, video_name)
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"❌ No se pudo abrir el video: {video_name}")
        continue

    keypoints_list = []
    frame_list = []
    print(f"🍓 Procesando: {video_name}")

    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        prev_keypoints = None
        start_index = None
        end_index = None
        movement_threshold = 5.0  # Adjust based on dataset (higher = less sensitive)

        frame_index = 0

        with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
            prev_keypoints = None
            start_index = None
            end_index = None
            movement_threshold = 5.0

            frame_index = 0

            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = holistic.process(frame_rgb)

                keypoints = np.zeros((TOTAL_KEYPOINTS, 3))

                valid_detection = False  # Track if we detected anything at all

                if results.pose_landmarks:
                    for i, lm in enumerate(results.pose_landmarks.landmark):
                        keypoints[i] = (lm.x, lm.y, lm.z)
                    valid_detection = True

                if results.left_hand_landmarks:
                    for i, lm in enumerate(results.left_hand_landmarks.landmark):
                        keypoints[POSE_LANDMARKS + i] = (lm.x, lm.y, lm.z)
                    valid_detection = True

                if results.right_hand_landmarks:
                    for i, lm in enumerate(results.right_hand_landmarks.landmark):
                        keypoints[POSE_LANDMARKS + HAND_LANDMARKS + i] = (lm.x, lm.y, lm.z)
                    valid_detection = True

                keypoints_list.append(keypoints)
                frame_list.append(frame)

                if valid_detection:
                    # Use this as fallback start if nothing else is found
                    if start_index is None:
                        start_index = frame_index

                    # Always update last valid detection
                    end_index = frame_index

                    # Optional: enhance with movement detection
                    if prev_keypoints is not None:
                        movement = calculate_movement(prev_keypoints, keypoints)
                        if movement > movement_threshold and start_index is None:
                            start_index = frame_index
                        if movement < movement_threshold:
                            end_index = frame_index

                    prev_keypoints = keypoints

                frame_index += 1

        cap.release()

    # ✅ Save keypoints
    if start_index is None or end_index is None:
        print(f"⚠️ No valid keypoints detected in: {video_name}")
        
    else:
        if keypoints_list:
            keypoints_array = np.array(keypoints_list)
            np.save(os.path.join(output_folder, video_name.replace(".mp4", ".npy")), keypoints_array)

        # ✅ Save start & end frames
        if start_index is not None:
            start_frame_path = os.path.join(frame_output_folder, f"{video_name}_start.jpg")
            cv2.imwrite(start_frame_path, frame_list[start_index])
            print(f"✅ Start frame saved: {start_frame_path}")

        if end_index is not None:
            end_frame_path = os.path.join(frame_output_folder, f"{video_name}_end.jpg")
            cv2.imwrite(end_frame_path, frame_list[end_index])
            print(f"✅ End frame saved: {end_frame_path}")

print("🎉 Proceso finalizado.")


9 videos encontrados. Procesando...
🍓 Procesando: Tyler-session-ASL_2008_05_29a-scene-49-700-717-camera1.mp4


I0000 00:00:1742563707.157235 1804077 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M4 Pro
I0000 00:00:1742563707.172113 1804077 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M4 Pro
W0000 00:00:1742563707.215994 1804888 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742563707.225405 1804883 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742563707.226496 1804893 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742563707.226570 1804884 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742563707.226570 1804883 in

✅ Start frame saved: ../data/hello/key_frames/Tyler-session-ASL_2008_05_29a-scene-49-700-717-camera1.mp4_start.jpg
✅ End frame saved: ../data/hello/key_frames/Tyler-session-ASL_2008_05_29a-scene-49-700-717-camera1.mp4_end.jpg
🍓 Procesando: Naomi-session-ASL_2008_08_13-scene-10-1822-1842-camera1.mp4


I0000 00:00:1742563707.685080 1804077 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M4 Pro
I0000 00:00:1742563707.700396 1804077 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M4 Pro
W0000 00:00:1742563707.738822 1804942 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742563707.747644 1804942 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742563707.748629 1804949 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742563707.748667 1804946 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742563707.749476 1804942 in

✅ Start frame saved: ../data/hello/key_frames/Naomi-session-ASL_2008_08_13-scene-10-1822-1842-camera1.mp4_start.jpg
✅ End frame saved: ../data/hello/key_frames/Naomi-session-ASL_2008_08_13-scene-10-1822-1842-camera1.mp4_end.jpg
🍓 Procesando: Dana-session-ASL_2007_05_24-scene-17-2369-2402-camera1.mp4


...

✅ Start frame saved: ../data/hello/key_frames/Liz-session-ASL_2008_02_01-scene-12-835-880-camera1.mp4_start.jpg
✅ End frame saved: ../data/hello/key_frames/Liz-session-ASL_2008_02_01-scene-12-835-880-camera1.mp4_end.jpg
🍓 Procesando: Brady-session-ASL_2011_06_14_Brady-scene-29-1865-1885-camera1.mp4


I0000 00:00:1742563711.814336 1804077 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M4 Pro
I0000 00:00:1742563711.828574 1804077 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M4 Pro
W0000 00:00:1742563711.868523 1805233 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742563711.878781 1805233 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742563711.879826 1805232 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742563711.879838 1805237 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742563711.880567 1805246 in

✅ Start frame saved: ../data/hello/key_frames/Brady-session-ASL_2011_06_14_Brady-scene-29-1865-1885-camera1.mp4_start.jpg
✅ End frame saved: ../data/hello/key_frames/Brady-session-ASL_2011_06_14_Brady-scene-29-1865-1885-camera1.mp4_end.jpg
🍓 Procesando: Naomi-session-ASL_2008_08_13-scene-17-1186-1205-camera1.mp4


I0000 00:00:1742563712.432207 1804077 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M4 Pro
I0000 00:00:1742563712.446880 1804077 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M4 Pro
W0000 00:00:1742563712.485442 1805274 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742563712.495295 1805274 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742563712.496367 1805278 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742563712.496388 1805283 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742563712.497113 1805279 in

✅ Start frame saved: ../data/hello/key_frames/Naomi-session-ASL_2008_08_13-scene-17-1186-1205-camera1.mp4_start.jpg
✅ End frame saved: ../data/hello/key_frames/Naomi-session-ASL_2008_08_13-scene-17-1186-1205-camera1.mp4_end.jpg
🎉 Proceso finalizado.
